# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as po
import requests
import xlsxwriter
import math

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [2]:
stocks = po.read_csv('./sp_500_stocks.csv')
stocks.head()

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV


## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [3]:
from secrets import IEX_CLOUD_API_TOKEN
# IEX_CLOUD_API_TOKEN = 'Tpk_059b97af715d417d9f49f50b51b1c448'

## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [25]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
# print(api_url)
data = requests.get(api_url).json()
# data

## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [13]:
price = data['latestPrice']
market_cap = data['marketCap']
print(price)
print(market_cap / 1000000000000)

159.28
2.658195743369


## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [19]:
my_columns = ['Ticker', 'Stock Price', 'Market Cap', 'Num. Shares to Buy']
final_df = po.DataFrame(columns=my_columns)
final_df.head()

,Ticker,Stock Price,Market Cap,Num. Shares to Buy


In [23]:
final_df.append(po.Series([symbol, price, market_cap, 'N/A'], index=my_columns), ignore_index=True)

,Ticker,Stock Price,Market Cap,Num. Shares to Buy
0,AAPL,159.28,2658195743369,N/A


## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [29]:
stock_df = po.DataFrame(columns=my_columns)
for stock in stocks['Ticker'][:10]:
    stock_data = requests.get(f'https://sandbox.iexapis.com/stable/stock/{stock}/quote/?token={IEX_CLOUD_API_TOKEN}').json()
    stock_df = stock_df.append(po.Series([stock, stock_data['latestPrice'], stock_data['marketCap'], 'N/A'], index=my_columns), ignore_index=True)

In [30]:
stock_df.head(5)

,Ticker,Stock Price,Market Cap,Num. Shares to Buy
0,A,185.460,55893234428,N/A
1,AAL,20.457,12892138002,N/A
2,AAP,200.950,12962415103,N/A
3,AAPL,157.940,2550570273542,N/A
4,ABBV,116.820,205381028812,N/A


## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [34]:
""" Separate the elements in lst into n-length size chunks. """
def chunks(lst, n):
    parts = []
    for i in range(0, len(lst), n):
        parts.append(lst[i : i + n])
    return parts

In [49]:
batch_data_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols=AAPL,F&types=quote&token={IEX_CLOUD_API_TOKEN}'
# print(batch_data_url)
batch_data = requests.get(batch_data_url).json()
# print(batch_data)
batch_data['AAPL']

{'quote': {'avgTotalVolume': 70268177,
  'calculationPrice': 'iexlasttrade',
  'change': 1.2,
  'changePercent': 0.00806,
  'close': 0,
  'closeSource': 'caliiffo',
  'closeTime': None,
  'companyName': 'Apple Inc',
  'currency': 'USD',
  'delayedPrice': None,
  'delayedPriceTime': None,
  'extendedChange': None,
  'extendedChangePercent': None,
  'extendedPrice': None,
  'extendedPriceTime': None,
  'high': 0,
  'highSource': None,
  'highTime': None,
  'iexAskPrice': 0,
  'iexAskSize': 0,
  'iexBidPrice': 0,
  'iexBidSize': 0,
  'iexClose': 160.35,
  'iexCloseTime': 1684779152212,
  'iexLastUpdated': 1646718971496,
  'iexMarketPercent': 0.013677915074691316,
  'iexOpen': 155.3,
  'iexOpenTime': 1635631078437,
  'iexRealtimePrice': 153.96,
  'iexRealtimeSize': 210,
  'iexVolume': 934556,
  'lastTradeTime': 1642362031444,
  'latestPrice': 161.17,
  'latestSource': 'IEX Last Trade',
  'latestTime': 'September 2, 2021',
  'latestUpdate': 1647791988147,
  'latestVolume': None,
  'low': 0,

In [51]:
ticker_groups = chunks(stocks['Ticker'], 100)
stringed_groups = []
for group in ticker_groups:
    stringed_groups.append(','.join(group))
    
stock_df = po.DataFrame(columns=my_columns)
for str_group in stringed_groups:
    batch_data_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={str_group}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    batch_data = requests.get(batch_data_url).json()
    for ticker in batch_data:
        # print(ticker)
        stock_data = batch_data[ticker]['quote']
        stock_df = stock_df.append(po.Series([ticker, stock_data['latestPrice'], stock_data['marketCap'], 'N/A'], index=my_columns), ignore_index=True)
# print(stringed_groups[0])
stock_df.head(5)

,Ticker,Stock Price,Market Cap,Num. Shares to Buy
0,A,181.460,56400597249,N/A
1,AAL,20.156,13033260555,N/A
2,AAP,208.040,12983202370,N/A
3,AAPL,159.810,2645076669787,N/A
4,ABBV,112.040,198307996170,N/A


## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

In [64]:
# Input takes in string; try-except to validate
portfolio_size = None
while type(portfolio_size) != float:
    try:
        portfolio_size = float(input('Enter value of your portfolio: '))
    except ValueError:
        print('Please enter a numerical value.')

Enter value of your portfolio: 10000000


In [69]:
position_size = portfolio_size / len(stock_df.index)

# print(position_size)
# num_aapl_shares = position_size / 150 # Where 150 is the price of AAPL stock

# Need to round DOWN to the nearest integer
# print(math.floor(num_aapl_shares))

for i in range(len(stock_df.index))[:5]:
    price = stock_df.loc[i, 'Stock Price']
    stock_df.loc[i, 'Num. Shares to Buy'] = math.floor(position_size / price)
    
stock_df.head(5)

,Ticker,Stock Price,Market Cap,Num. Shares to Buy
0,A,181.460,56400597249,109
1,AAL,20.156,13033260555,982
2,AAP,208.040,12983202370,95
3,AAPL,159.810,2645076669787,123
4,ABBV,112.040,198307996170,176


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

## Saving Our Excel Output

Saving our Excel file is very easy: